# Tilores

本指南介绍了如何开始使用 [Tilores](/docs/integrations/providers/tilores) 工具。
一个更复杂的示例，您可以查看我们的 [客户洞察聊天机器人示例](https://github.com/tilotech/identity-rag-customer-insights-chatbot)。

## 概览

### 集成详情

| Class | Package | 可序列化 | JS 支持 | 最新版本 |
| :--- | :--- | :---: | :---: | :---: |
| TiloresTools | [tilores-langchain](https://pypi.org/project/tilores-langchain/) | ❌ | ❌ |  ![PyPI - Version](https://img.shields.io/pypi/v/tilores-langchain?style=flat-square&label=%20) |

## 设置

集成需要以下包：

In [72]:
%pip install --quiet -U tilores-langchain langchain

Note: you may need to restart the kernel to use updated packages.


### 凭证

要访问 Tilores，你需要[创建并配置一个实例](https://app.tilores.io)。如果你想先试用 Tilores，可以使用[只读演示凭证](https://github.com/tilotech/identity-rag-customer-insights-chatbot?tab=readme-ov-file#1-configure-customer-data-access)。

In [73]:
import os

os.environ["TILORES_API_URL"] = "<api-url>"
os.environ["TILORES_TOKEN_URL"] = "<token-url>"
os.environ["TILORES_CLIENT_ID"] = "<client-id>"
os.environ["TILORES_CLIENT_SECRET"] = "<client-secret>"

## 实例化

这里我们展示如何实例化 Tilores 工具的实例：

In [74]:
from tilores import TiloresAPI
from tilores_langchain import TiloresTools

tilores = TiloresAPI.from_environ()
tilores_tools = TiloresTools(tilores)
search_tool = tilores_tools.search_tool()
edge_tool = tilores_tools.edge_tool()

## 调用

`tilores_search` 工具的参数取决于 Tilores 中[配置的 schema](https://docs.tilotech.io/tilores/schema/)。以下示例将使用带有生成数据的演示实例的 schema。

### [直接带参数调用](/docs/concepts/tools)

以下示例搜索居住在柏林的 Sophie Müller。Tilores 数据包含多个以此为名的人，并返回他们已知的电子邮件地址和电话号码。

In [75]:
result = search_tool.invoke(
    {
        "searchParams": {
            "name": "Sophie Müller",
            "city": "Berlin",
        },
        "recordFieldsToQuery": {
            "email": True,
            "phone": True,
        },
    }
)
print("Number of entities:", len(result["data"]["search"]["entities"]))
for entity in result["data"]["search"]["entities"]:
    print("Number of records:", len(entity["records"]))
    print(
        "Email Addresses:",
        [record["email"] for record in entity["records"] if record.get("email")],
    )
    print(
        "Phone Numbers:",
        [record["phone"] for record in entity["records"] if record.get("phone")],
    )

Number of entities: 3
Number of records: 3
Email Addresses: ['s.mueller@newcompany.de', 'sophie.mueller@email.de']
Phone Numbers: ['30987654', '30987654', '30987654']
Number of records: 5
Email Addresses: ['mueller.sophie@uni-berlin.de', 'sophie.m@newshipping.de', 's.mueller@newfinance.de']
Phone Numbers: ['30135792', '30135792']
Number of records: 2
Email Addresses: ['s.mueller@company.de']
Phone Numbers: ['30123456', '30123456']


如果我们想了解来自第一个实体的记录是如何关联的，我们可以使用 `edge_tool`。请注意，Tilores 实体解析引擎已自动找出这些记录之间的关联。有关更多详细信息，请参阅 [edge 文档](https://docs.tilotech.io/tilores/rules/#edges)。

In [76]:
edge_result = edge_tool.invoke(
    {"entityID": result["data"]["search"]["entities"][0]["id"]}
)
edges = edge_result["data"]["entity"]["entity"]["edges"]
print("Number of edges:", len(edges))
print("Edges:", edges)

Number of edges: 7
Edges: ['e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6:f2g3h4i5-j6k7-l8m9-n0o1-p2q3r4s5t6u7:L1', 'e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6:g3h4i5j6-k7l8-m9n0-o1p2-q3r4s5t6u7v8:L4', 'e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6:f2g3h4i5-j6k7-l8m9-n0o1-p2q3r4s5t6u7:L2', 'f2g3h4i5-j6k7-l8m9-n0o1-p2q3r4s5t6u7:g3h4i5j6-k7l8-m9n0-o1p2-q3r4s5t6u7v8:L1', 'f2g3h4i5-j6k7-l8m9-n0o1-p2q3r4s5t6u7:g3h4i5j6-k7l8-m9n0-o1p2-q3r4s5t6u7v8:L4', 'e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6:g3h4i5j6-k7l8-m9n0-o1p2-q3r4s5t6u7v8:L1', 'e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6:f2g3h4i5-j6k7-l8m9-n0o1-p2q3r4s5t6u7:L4']


### [使用 ToolCall 调用](/docs/concepts/tools)

我们也可以使用模型生成的 ToolCall 来调用工具，在这种情况下会返回一个 ToolMessage：

In [77]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {
        "searchParams": {
            "name": "Sophie Müller",
            "city": "Berlin",
        },
        "recordFieldsToQuery": {
            "email": True,
            "phone": True,
        },
    },
    "id": "1",
    "name": search_tool.name,
    "type": "tool_call",
}
search_tool.invoke(model_generated_tool_call)

ToolMessage(content='{"data": {"search": {"entities": [{"id": "9601cf3b-e85f-46ab-aaa8-ffb8b46f1c5b", "hits": {"c3d4e5f6-g7h8-i9j0-k1l2-m3n4o5p6q7r8": ["L1"]}, "records": [{"email": "", "phone": "30123456"}, {"email": "s.mueller@company.de", "phone": "30123456"}]}, {"id": "03da2e11-0aa2-4d17-8aaa-7b32c52decd9", "hits": {"e1f2g3h4-i5j6-k7l8-m9n0-o1p2q3r4s5t6": ["L1"], "g3h4i5j6-k7l8-m9n0-o1p2-q3r4s5t6u7v8": ["L1"]}, "records": [{"email": "s.mueller@newcompany.de", "phone": "30987654"}, {"email": "", "phone": "30987654"}, {"email": "sophie.mueller@email.de", "phone": "30987654"}]}, {"id": "4d896fb5-0d08-4212-a043-b5deb0347106", "hits": {"j6k7l8m9-n0o1-p2q3-r4s5-t6u7v8w9x0y1": ["L1"], "l8m9n0o1-p2q3-r4s5-t6u7-v8w9x0y1z2a3": ["L1"], "m9n0o1p2-q3r4-s5t6-u7v8-w9x0y1z2a3b4": ["L1"], "n0o1p2q3-r4s5-t6u7-v8w9-x0y1z2a3b4c5": ["L1"]}, "records": [{"email": "mueller.sophie@uni-berlin.de", "phone": ""}, {"email": "sophie.m@newshipping.de", "phone": ""}, {"email": "", "phone": "30135792"}, {"email":

## 链式调用

我们可以先将我们的工具绑定到一个 [工具调用模型](/docs/how_to/tool_calling/)，然后进行调用，从而实现工具的链式调用：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [ ]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# specifying tool_choice will force the model to call this tool.
llm_with_tools = llm.bind_tools([search_tool], tool_choice=search_tool.name)

llm_chain = prompt | llm_with_tools


@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = search_tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


tool_chain.invoke("Tell me the email addresses from Sophie Müller from Berlin.")

## API 参考

有关 Tilores 所有功能和配置的详细文档，请访问官方文档：https://docs.tilotech.io/tilores/